### Przeciążone sygnatury

Funkcje w Pythonie mogą przyjmować różne kombinacje argumentów. Dekorator `typing.overload` pozwala na adnotacje dla tych kombincji. Jest to ważne gdy typ zwracanej wartości zależy od typu dwóch lub więcej parametrów.

In [1]:
from functools import reduce
from operator import add

from typing import Iterable, Union, TypeVar, overload

_T = TypeVar("_T")
_S = TypeVar("_S")


@overload
def mysum(__it: Iterable[_T], /) -> Union[_T, int]: ...  # 1 sygnatura
@overload
def mysum(__it: Iterable[_T], /, start: _S) -> Union[_T, _S]: ...  # 2 sygnatura
def mysum(__it, /, start=0):
    return reduce(add, __it, start)

Wielokropek `...` nie ma żadnej funkcji oprócz spełnienia wymogu syntaktycznego dla ciała funkcji, podobnie do `pass`. Dwa początkowe podkreślenia w `__it` stanowią konwencję z PEP 484 dla argumentów wyłącznie pozycyjnych, która jest wymuszana przez Mypy. 

Pierwsza sygnatura obsługuje prosty przypadek gdy typem zwracanego wyniku jest `_T` - typ elementów zwracanych przez `__it`, albo `int` jeśli obiekt iterowalny jest pusty bo domyślną wartością parametru `start` jest 0.

Druga sygnatura obsługuje przypadek gdy podany jest argument `start`, który może być dowolnego typu `_S`, więc typem zwracanego wyniku jest `_T` lub `_S`. 

Sygnatura faktycznej implementacji nie ma żadnych wskazówek do typów.

In [2]:
mysum([1, 2, 3])

6

In [3]:
mysum(["a", "b", "c"], start="x")

'xabc'

### Odczytywanie wskazówek do typów w czasie wykonywania programu

In [4]:
def clip(text: str, max_len: int = 80) -> str: ...

In [5]:
clip.__annotations__

{'text': str, 'max_len': int, 'return': str}

Klucz 'return' jest mapowany na wskazówek dla typu zwracanej wartości po symbolu '->'. 

Adnotacje są przetwarzane przez interpreter w czasie importu, wtedy gdy przetwarzane są też domyślne wartości parametrów. Dlatego właśnie wartości w adnotacjach są pythonicznymi klasami `str` i `int` a nie łańcuchami znaków 'str' i 'int'.

- Importowanie modułów zużywa więcej zasobów procesora i pamięci, gdy używanych jest wiele wskazówek do typów.
- Odwoływanie się do typów jeszcze niezdefiniowanych wymaga użycia łańcuchów tesktowych zamiast faktycznych typów.

In [6]:
class Rectangle:
    def __init__(self, width: float, height: float) -> None:
        self.width = width
        self.height = height

    def stretch(self, factor: float) -> "Rectangle":
        return Rectangle(width=self.width * factor, height=self.height * factor)

Ponieważ obiekt klasy nie jest jeszcze zdefiniowany, dopóki Python nie przetworzy zawartości klasy, wskazówki dla typów muszą używać nazwy klasy w postaci łańcucha. Jednakże w czasie wykonywania programu gdy odczytamy adnotacje, uzyskamy również łańcuch tekstowy zamiast faktyczny typ.

In [7]:
Rectangle.stretch.__annotations__

{'factor': float, 'return': 'Rectangle'}

In [8]:
import inspect

inspect.get_annotations(Rectangle.stretch)

{'factor': float, 'return': 'Rectangle'}

In [9]:
import typing

typing.get_type_hints(Rectangle.stretch)

{'factor': float, 'return': __main__.Rectangle}

Od wersji Pythona 3.7 adnotacje są obsługiwane zawsze w psotaci tesktowej jeżeli moduł zaweira import `from __future__ import annotations`. 

In [11]:
from __future__ import annotations


class Rectangle:
    def __init__(self, width: float, height: float) -> None:
        self.width = width
        self.height = height

    def stretch(self, factor: float) -> Rectangle:
        return Rectangle(width=self.width * factor, height=self.height * factor)


Rectangle.stretch.__annotations__

{'factor': 'float', 'return': 'Rectangle'}

### Implementowanie klasy generycznej

- Typ generyczny (ogólny) - typ deklarowany z jedną lub kilkoma zmiennymi typów:
  - `LottoBlower[T]`,
  - `abc.Mapping[KT, VT]`.
- Formalny parametr typu - zmienne dla typów, które pojawiają się w deklaracji typu generycznego:
  - `KT` i `VT` w poprzednim przykładzie.
- Typ sparametryzowany - typ zadeklarowany z faktycznymi parametrami dla typów:
  - `LottoBlower[int]`,
  - `abc.Mapping[str, float]`.
- Faktyczny parametr typu - faktyczne typy podawane jako parametry, gdy deklarowany typ jest sparametryzowany:
  - `int` w `LottoBlower[int]`.

### Wariancja

Wyobraźmy sobie, że szkolny bufet ma zasadę, że można instalować tylko dozowniki do soków owocowych. Generyczne dozowniki nie są dozwolone. Toto jak możemy to osiągnąć.

In [14]:
from typing import TypeVar, Generic


class Beverage:
    """Dowolny napój."""


class Juice(Beverage):
    """Dowolny sok owocowy."""


class OrangeJuice(Juice):
    """Sok pomarańczowy."""


T = TypeVar("T")


class BeverageDispenser(Generic[T]):
    """Dozowonik parametryzowany według typu napoju."""

    def __init__(self, beverage: T) -> None:
        self.beverage = beverage

    def dispense(self) -> T:
        return self.beverage


def install(dispenser: BeverageDispenser[Juice]) -> None:
    """Zainstaluj dozownik soku owocowego."""

`Beverage`, `Juice` i `OrangeJuice` tworzą hierarchię typów dla napojów. `BeverageDispenser` jest sparametryzowany typem napoju. Funkcja `install()` jest funkcją globalną w module. Jej wskazówka dla typu wymusza regułę, że dopuszczalny jest tylko dozownik soków.

In [15]:
juice_dispenser = BeverageDispenser(Juice())
install(juice_dispenser)  # OK

In [ ]:
beverage_dispenser = BeverageDispenser(Beverage())
install(beverage_dispenser)  # Błąd

# Argument of type "BeverageDispenser[Beverage]" cannot be assigned to parameter
# "dispenser" of type "BeverageDispenser[Juice]" in function "install"
# "BeverageDispenser[Beverage]" is incompatible with "BeverageDispenser[Juice]"
# Type parameter "T@BeverageDispenser" is invariant, but "Beverage" is not the same as "Juice"

Dozownik obsługujący dowolny typ `Beverage` nie jest dopuszczalny, ponieważ bufet wymaga dozownika wyspecjalizowanego w podawaniu `Juice`.

In [16]:
orange_juice_dispenser = BeverageDispenser(OrangeJuice())
install(orange_juice_dispenser)  # Zaskakujący błąd!

Dozownik specjalizujący się w `OrangeJuice` również nie jest dozwolony. Dopuszczalny jest jedynie `BeverageDispenser[Juice]`. W żargonie związanym z typowaniem mówimy, że `BeverageDispenser(Generic[T])` jest typem bezwariantowym (niezmienniczym), gdy `BeverageDispenser[OrangeJuice]` nie jest kompatybilny z `BeverageDispenser[Juice]` - pomimo faktu, że `OrangeJuice` jest podtypem `Juice`. Typy zmiennych kolekcji w Pythonie - takie jak `list`, `set` są bezwariantowe.

### Kowariancja

Jeżeli chcemy być bardziej elastyczni i modelować dozownik jako klasę generyczną, która może przyjmować jakiś typ napoju a także jego podtypy, musimy uczynić go kowariantnym.

In [22]:
T_co = TypeVar("T_co", covariant=True)


class BeverageDispenser(Generic[T_co]):
    def __init__(self, beverage: T_co) -> None:
        self.beverage = beverage

    def dispense(self) -> T_co:
        return self.beverage


def install(dispenser: BeverageDispenser[Juice]) -> None:
    """Zainstaluj dozownik soku owocowego."""

In [23]:
juice_dispenser = BeverageDispenser(Juice())
install(juice_dispenser)  # OK

In [24]:
orange_juice_dispenser = BeverageDispenser(OrangeJuice())
install(orange_juice_dispenser)  # OK

In [25]:
beverage_dispenser = BeverageDispenser(Beverage())
install(beverage_dispenser)  # Błąd

To właśnie jest kowariancja: relacja podtypu w sparametryzowanych dozownikach określona jest w tym samym kierunku, co relacja podtypu dla parametrów typów.

### Kontrawariancja

Zamodelujemy regułę bufetu dotyczącą pojemników na śmieci. Załóżmy, że jedzenie i napoje są serwowane w biodegradowalnych naczyniach, a resztki i sztućce jednorazowe również są biodegradowalne. Pojemniki na śmieci muszą być odpowiednie dla odpadów biodegradowalnych.

In [27]:
class Refuse:
    """Dowolne śmieci."""


class Biodegradable(Refuse):
    """Biodegradowalne śmieci."""


class Compostable(Biodegradable):
    """Kompostowalne śmieci."""


T_contra = TypeVar("T_contra", contravariant=True)


class TrashCan(Generic[T_contra]):
    def put(self, refuse: T_contra) -> None:
        """Przechowaj śmieci."""


def deploy(trash_can: TrashCan[Biodegradable]) -> None:
    """Dostarcz pojemnik na biodegradowalne śmieci."""

In [28]:
bio_can: TrashCan[Biodegradable] = TrashCan()
deploy(bio_can)  # OK

In [29]:
trash_can: TrashCan[Refuse] = TrashCan()
deploy(trash_can)  # OK

In [30]:
compost_can: TrashCan[Compostable] = TrashCan()
deploy(compost_can)  # Błąd

# Argument of type "TrashCan[Compostable]" cannot be assigned to parameter "trash_can" of type "TrashCan[Biodegradable]" in function "deploy"
#   "TrashCan[Compostable]" is incompatible with "TrashCan[Biodegradable]"
#     Type parameter "T_contra@TrashCan" is contravariant, but "Compostable" is not a supertype of "Biodegradable"
#       "Biodegradable" is incompatible with "Compostable"